# Interpolate big data time series in native pySpark
Databricks pyspark notebook showing how big data time series can be interpolated in spark without using UDF's

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
import timeit

In [0]:
nsensors = 10
ntimestamps = 1000
resample_interval = 60*60  # Resample interval size in seconds
interval_from = '2021-01-01 00:00:00'
interval_to = '2021-01-07 00:00:00'

# Create test data

In [0]:
# Per sensor create ntimestamps random time values between interval_from and interval_to
# Take a sinus based function to calculate a value (take different magnitude and period per sensor)
df_sensors = spark.range(nsensors).selectExpr('id+1 as SensorId')
df_time = spark.range(ntimestamps).selectExpr('id as Range')
df_test = (df_sensors.join(df_time, how='full')
           .withColumn('UnixTimestamp', F.expr(f"round(rand()*(unix_timestamp('{interval_to}')-unix_timestamp('{interval_from}'))+unix_timestamp('{interval_from}'))"))
           .withColumn('Timestamp', F.expr("to_timestamp(UnixTimestamp)"))
           .withColumn('Value', F.expr(f"SensorId * sin(2*3.14/SensorId*(UnixTimestamp - unix_timestamp('{interval_from}'))/(unix_timestamp('{interval_to}')-unix_timestamp('{interval_from}')))"))
           .drop('UnixTimestamp')
           .drop('Range')
           # Make sure no duplicate rows
           .dropDuplicates(['SensorId', 'Timestamp'])
          )

In [0]:
df_test.persist()

Out[5]: DataFrame[SensorId: bigint, Timestamp: timestamp, Value: double]

In [0]:
df_test.count()

Out[6]: 9995

# Resample + interpolate pyspark

In [0]:
df_pyspark = (
      df_test
      # Get timestamp and value of previous measurement
      .selectExpr(
        "SensorId",
        "LAG(Timestamp) OVER (PARTITION BY SensorId ORDER BY Timestamp ASC) as PreviousTimestamp",
        "Timestamp as NextTimestamp",
        "LAG(Value) OVER (PARTITION BY SensorId ORDER BY Timestamp ASC) as PreviousValue",
        "Value as NextValue",
      )
      # To determine resample interval round up start and round down end timeinterval to nearest interval boundary
      .withColumn("PreviousTimestampRoundUp", F.expr(f"to_timestamp(ceil(unix_timestamp(PreviousTimestamp)/{resample_interval})*{resample_interval})"))
      .withColumn("NextTimestampRoundDown", F.expr(f"to_timestamp(floor(unix_timestamp(NextTimestamp)/{resample_interval})*{resample_interval})"))
      # Make sure we don't get any negative intervals (whole interval is within resample interval)
      .filter("PreviousTimestampRoundUp<=NextTimestampRoundDown")
      # Create resampled time axis by creating all "interval" timestamps between previous and next timestamp.
      .withColumn("Timestamp", F.expr(f"explode(sequence(PreviousTimestampRoundUp, NextTimestampRoundDown, interval {resample_interval} second)) as Timestamp"))
      # Sequence has inclusive boundaries for both start and stop. Filter out duplicate value if original timestamp is exactly a boundary.
      .filter("Timestamp<NextTimestamp")
      # Interpolate value between previous and next
      .selectExpr(
        "SensorId",
        "Timestamp", 
        """(unix_timestamp(Timestamp)-unix_timestamp(PreviousTimestamp))
            /(unix_timestamp(NextTimestamp)-unix_timestamp(PreviousTimestamp))
            *(NextValue-PreviousValue) 
            +PreviousValue
            as Value"""
      )
)

In [0]:
%%timeit  -n 1 -r 1
df_pyspark.count()

576 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

In [0]:
display(df_test.filter("SensorId = 1"))

SensorId,Timestamp,Value
1,2021-01-06T04:27:29.000+0000,-0.7548538189891042
1,2021-01-01T03:23:06.000+0000,0.14708800812677128
1,2021-01-03T03:55:00.000+0000,0.7691110865899089
1,2021-01-05T10:51:48.000+0000,-0.9986503685171155
1,2021-01-02T13:51:21.000+0000,0.9967906183976832
1,2021-01-02T17:53:22.000+0000,0.9673981859421079
1,2021-01-04T00:37:30.000+0000,-0.025661473624987472
1,2021-01-01T12:30:53.000+0000,0.5190857366207262
1,2021-01-02T08:16:22.000+0000,0.9866889022069436
1,2021-01-06T01:50:13.000+0000,-0.8247391518340605


In [0]:
display(df_pyspark.filter("sensorId=1"))

SensorId,Timestamp,Value
1,2021-01-01T01:00:00.000+0000,0.04359603912961019
1,2021-01-01T02:00:00.000+0000,0.08711136958637623
1,2021-01-01T03:00:00.000+0000,0.13045056645860106
1,2021-01-01T04:00:00.000+0000,0.17355117702015466
1,2021-01-01T05:00:00.000+0000,0.21633155343868873
1,2021-01-01T06:00:00.000+0000,0.25868957176280644
1,2021-01-01T07:00:00.000+0000,0.3005576154857251
1,2021-01-01T08:00:00.000+0000,0.34185085500721046
1,2021-01-01T09:00:00.000+0000,0.38249715351897406
1,2021-01-01T10:00:00.000+0000,0.4224155338213948
